In [1]:
import numpy as np

# Stepp9 함수를 더 편리하게
## 파이썬 함수로 이용하기
그 동안 클래스로 구현되었던 func들을 파이썬 함수로 작성해서 활용성을 높이자!

In [5]:
# 반복문으로 새로 구현
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    funcs = [self.creator]     
    while funcs :              
      f = funcs.pop()          
      x, y = f.input, f.output 
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator) 

In [6]:
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)  
    output = Variable(y)

    output.set_creator(self) 
    
    self.input = input
    self.output= output 
    return output

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.input.data
    gx = 2 * x * gy
    return gx

class Exp(Function):
  def forward(self, x):
    y = np.exp(x)
    return y

  def backward(self, gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

In [7]:
# 함수로 작성해서 더 편리하게!
def square(x):
  return Square()(x)

def exp(x):
  return Exp()(x)

In [9]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


## 9.2 backward 메서드 간소화

In [10]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    # 이 if 문으로 grad를 자동으로 1로 줄 수 있음
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]     
    while funcs :              
      f = funcs.pop()          
      x, y = f.input, f.output 
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator) 

In [11]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))

y.backward()
print(x.grad)

3.297442541400256


## 9.3 ndarray만 취급하기
사용자가 ndarray가 아닌 다른 인스턴스를 사용했을 때 오류를 일으키도록.

In [12]:
class Variable:
  def __init__(self, data):
    # data의 type이 np.ndarray가 아니면 오류를 발생
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}는 지원하지 않습니다.'.format(type(data)))
    
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    # 이 if 문으로 grad를 자동으로 1로 줄 수 있음
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]     
    while funcs :              
      f = funcs.pop()          
      x, y = f.input, f.output 
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator) 

In [13]:
x = Variable(np.array(1.0)) # ok
x = Variable(None) # ok

x = Variable(1.0) # 오류 발생!

TypeError: ignored

In [14]:
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim) # <class 'numpy.ndarray'> 1
print(type(y)) #<class 'numpy.ndarray'>

x = np.array(1.0)
y = x ** 2
print(type(x), x.ndim) # <class 'numpy.ndarray'> 0
print(type(y)) # numpy의 성질 때문에 0차원의 scalar를 제곱하면 numpy.float으로 변환됨.
# 이럴경우, np.array로 다시 만들어 줄 함수 필요.

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>
<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [15]:
def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

In [16]:
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)  
    # y가 scalar인지 쉽게 확인할 수 있다!
    # foward의 결과가 항상 ndarray 인스턴스가 되도록 보장!
    output = Variable(as_array(y))
    output.set_creator(self) 
    
    self.input = input
    self.output= output 
    return output

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()